In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/housesalesprediction/kc_house_data.csv")
df.head()

In [ ]:
df.isnull().sum() #no missing data

In [ ]:
df.info()

In [ ]:
df.describe()

# EDA

In [ ]:
plt.figure(figsize=(12,7))
sns.distplot(df["price"])
#The range goes up to 8e6, but the frequency on those values is very low, meaning those values are outliers

In [ ]:
sns.countplot(df["bedrooms"])

In [ ]:
df.corr().price.sort_values()

In [ ]:
plt.figure(figsize=(12,7))
sns.scatterplot(x="price",y="sqft_living", data=df)

In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(x="bedrooms",y="price",data=df)

In [ ]:
plt.figure(figsize=(12,7))
sns.scatterplot(x="price",y="long", data=df)

In [ ]:
plt.figure(figsize=(12,7))
sns.scatterplot(x="price",y="lat", data=df)

In [ ]:
plt.figure(figsize=(12,7))
sns.scatterplot(x="long",y="lat", data=df, hue="price") 

In [ ]:
df.sort_values("price", ascending=False).head(30)

In [ ]:
len(df)*0.01

In [ ]:
no_outliers=df.sort_values("price", ascending=False).iloc[217:]

In [ ]:
plt.figure(figsize=(12,7))
sns.scatterplot(x="long",y="lat", data=no_outliers, edgecolor=None,palette="RdYlGn", alpha=
                0.3, hue="price") 

In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(x="waterfront",y="price",data=df)

# Removing the outliers might improve the fit of our model, and so we will make a model only for house up to 2.5 mi dollars

In [ ]:
plt.figure(figsize=(12,7))
sns.distplot(no_outliers["price"])

# Feature Engineering

In [ ]:
df=no_outliers

In [ ]:
df.drop("id", axis=1, inplace=True)

In [ ]:
df["date"]=pd.to_datetime(df["date"])
df["year"]=df["date"].apply(lambda date: date.year)
df.drop("date",axis=1,inplace=True)

In [ ]:
df["zipcode"].value_counts() #This feature could be reengineered into something more useful with domain knowledge
df.drop("zipcode",axis=1,inplace=True)

# Since we have year of sale, year of construction and year of renovation, we will engineer these into two more consistent features: Years between building and renovation and years between renovation and sale

In [ ]:
df["yr_renovated"].value_counts()

In [ ]:
df.iloc[1]["yr_renovated"]

In [ ]:
not_renov=list(df["yr_renovated"]==0)
yr_renov=[]
for i in range(0,len(not_renov)):
    if not_renov[i]:
        yr_renov.append(df.iloc[i]["yr_built"])
    else:
        yr_renov.append(df.iloc[i]["yr_renovated"])
df["yr_renovated"]=yr_renov
df["built_to_renov"]=df["yr_renovated"]-df["yr_built"]
df["renov_to_sold"]=df["year"]-df["yr_renovated"]
df.drop(["yr_renovated","year"],axis=1,inplace=True)

# We will leave "sqft_basement" as is since the number 0 indicates the absence of a basement

In [ ]:
df["sqft_basement"].value_counts()

In [ ]:
df.drop(["lat","long"],axis=1,inplace=True)

In [ ]:
df.head()

# Modeling

In [ ]:
X = df.drop('price',axis=1)
y = df['price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
X_train.shape

In [ ]:
model = Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(3,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train,y=y_train.values,validation_data=(X_test,y_test.values),
          batch_size=128,epochs=400)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [ ]:
predictions=model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
df["price"].describe()

In [ ]:
explained_variance_score(y_test,predictions)

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_test,predictions)
plt.plot(y_test,y_test,"r")